# TruBot in Dashboard

This notebook adds trubot to the dashboard so it can be queried there. See also `appui_example.ipynb` that creates the same but for interacting with the app inside a jupyter notebook.

# This work is ongoing.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY",
)

In [ ]:
from pprint import PrettyPrinter

# Imports from langchain to build app:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
import numpy as np

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import feedback
from trulens_eval import FeedbackMode
from trulens_eval import Select
from trulens_eval import TP
from trulens_eval import Tru
from trulens_eval.utils.langchain import WithFeedbackFilterDocuments

pp = PrettyPrinter()

# Tru object manages the database of apps, records, and feedbacks; and the
# dashboard to display these.
tru = Tru()

# Start the dasshboard. If you running from github repo, you will need to adjust
# the path the dashboard streamlit app starts in by providing the _dev argument.
tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

# If needed, you can reset the trulens_eval dashboard database by running the
# below line:

# tru.reset_database()

In [ ]:
model_name = "gpt-3.5-turbo"
app_id = "TruBot"

# Embedding for vector db.
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

# Local pinecone alternative. Requires precomputed 'hnswlib_truera' folder.
from langchain.vectorstores import DocArrayHnswSearch

def get_doc_search():
    # We need to create this object in the thread in which it is used so we
    # wrap it in this function for later usage.

    docsearch = DocArrayHnswSearch.from_params(
        embedding=embedding,
        work_dir='hnswlib_trubot',
        n_dim=1536,
        max_elements=1024
    )

    return docsearch

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=256)

# Construct feedback functions.

# API endpoints for models used in feedback functions:
# hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
# f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will evaluate feedback on main app input and main app output.

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()
# By default this will evaluate feedback on main app input and main app output.

# Question/statement relevance between question and each context chunk.
f_qs_relevance = feedback.Feedback(openai.qs_relevance).on_input().on(
    Select.Record.app.combine_docs_chain._call.args.inputs.input_documents[:].page_content
).aggregate(np.min)
# First feedback argument is set to main app input, and the second is taken from
# the context sources as passed to an internal `combine_docs_chain._call`.

all_feedbacks = [
    #f_lang_match, 
    #f_qa_relevance, f_qs_relevance
]

In [ ]:
def v1_new_conversation(feedback_mode=FeedbackMode.WITH_APP):
    """
    Create a langchain app for a new conversation with a question-answering bot.

    Feedback_mode controls when feedback is evaluated:

    - FeedbackMode.WITH_APP -- app will wait until feedback is evaluated before
      returning from calls.

    - FeedbackMode.WITH_APP_THREAD -- app will return from calls and evaluate
      feedback in a new thread.

    - FeedbackMode.DEFERRED -- app will return and a separate runner thread (see
      usage later in this notebook) will evaluate feedback.
    """

    # Blank conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    docsearch = get_doc_search()

    # Context retriever.
    retriever = docsearch.as_retriever()

    # Conversational app puts it all together.
    app = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Trulens instrumentation.
    tc = Tru().Chain(
        app_id=f"{app_id}/v1",
        chain=app,
        feedbacks=all_feedbacks,
        feedback_mode=feedback_mode, 
    )

    return tc